Using command kaggle we download the dataset from kaggle website (further api doc can be found on kaggle website).

In [ ]:
#!python3 -m pip install kaggle
import kaggle
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

The downloaded files come in a .zip format so we extract them with zipfile into our repo.

In [2]:
import zipfile

zip_ref = zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall() # extract file to dir
zip_ref.close() # close file

Finally we can read the data using pandas read_csv.
Here we update values 'Not Available','unknown' into null, to clean our data.

In [4]:
import pandas as pd

df = pd.read_csv('orders.csv',na_values=['Not Available','unknown'])
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

Next step is to rename the columns into cleaner names using replace, changing spaces to underscores and making everything lowercase.

In [5]:
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(' ','_')
df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


We create 3 new columns, discount, sale_price and profit.

In [6]:
df['discount']=df['list_price']*df['discount_percent']*.01
df['sale_price']= df['list_price']-df['discount']
df['profit']=df['sale_price']-df['cost_price']
df.head()


,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-1.0


We convert order date from object data type to datetime


In [7]:
df['order_date']=pd.to_datetime(df['order_date'],format="%Y-%m-%d")
df.dtypes

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

We dropped unnecesary columns to save storage space.

In [8]:
#drop cost price list price and discount percent columns
#df.drop(columns=['list_price','cost_price','discount_percent'],inplace=True)
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,1.0,19.0,-1.0


After cleaning our data we are ready to connect to our database to create our table and upload the rows. Using sqlAlchemy we create a connection to our local postgres database.

In [13]:
#!pip install sqlalchemy


import sqlalchemy
from sqlalchemy import create_engine

engine = create_engine("postgresql://patricioguerra:@localhost:5432/master")
conn=engine.connect()

df.to_sql('df_orders', con=conn , index=False, if_exists = 'append')

994

Since we only have acces to Tableau Public we cannot conect directly from our postgres db (functionality available for paid users). In this case we create a sql query to extract only the necessary data using python, sqlalchemy and postgresql.

Finally we extract it into a .xlsx ready to be used by Tableau Public.

In [64]:
from sqlalchemy import text
from pandas import DataFrame


query = text(
    "with cte as ( select sub_category,extract(year from order_date) as order_year, sum(profit * quantity) as profit from df_orders group by sub_category,order_year) , cte2 as ( select sub_category , sum(case when order_year=2022 then profit else 0 end) as profit_2022 , sum(case when order_year=2023 then profit else 0 end) as profit_2023 from cte group by sub_category ) select *, (profit_2023 - profit_2022) as profit_growth from cte2 order by profit_growth desc"
    )

'''
QUERY in SQL FORMAT FOR READING PORPOUSES

with cte as (
select sub_category,extract(year from order_date) as order_year,
sum(profit * quantity) as profit
from df_orders
group by sub_category,order_year
	)
, cte2 as (
select sub_category
, sum(case when order_year=2022 then profit else 0 end) as profit_2022
, sum(case when order_year=2023 then profit else 0 end) as profit_2023
from cte
group by sub_category
)
select *, (profit_2023 - profit_2022) as profit
from  cte2
order by profit desc;

'''


with engine.connect() as conn:
    result = conn.execute(query)

df_query = DataFrame(result.fetchall())
df_query.columns = result.keys()

df_query

df_query.to_excel('subcat profit growth 22 23.xlsx', sheet_name='Data')


Additional queries created, not used because of Tableau Public limitations:


--Top 10 highest reveue generating products 
select product_id,sum(sale_price * quantity) as sales
from df_orders
group by product_id
order by sales desc
limit 10;




--Top 5 highest selling products in each region
-- create cte with region, id and revenue grouped by region and product
with cte as (
select region,product_id,sum(sale_price*quantity) as sales
from df_orders
group by region,product_id)
select * from (
--then add rownumber to filter the top 5 partitioned by region
select *
, row_number() over(partition by region order by sales desc) as rn
from cte) A
where rn<=5



--Month over month growth comparison for 2022 and 2023 sales eg : jan 2022 vs jan 2023
with cte as (
select date_part('year', order_date) as order_year,date_part('month',order_date) as order_month,
sum(sale_price * quantity) as sales
from df_orders
group by order_year, order_month
--order by year(order_date),month(order_date)
	)
select order_month
, sum(case when order_year=2022 then sales else 0 end) as sales_2022
, sum(case when order_year=2023 then sales else 0 end) as sales_2023
from cte 
group by order_month
order by order_month





--for each category which month had highest sales 
with cte as (
select category,TO_CHAR(order_date, 'yyyyMM') as order_year_month
, sum(sale_price*quantity) as sales 
from df_orders
group by category,order_year_month
--order by category,format(order_date,'yyyyMM')
)
select * from (
select *,
row_number() over(partition by category order by sales desc) as rn
from cte
) a
where rn=1